# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import gmaps.datasets

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_path='city_data.csv'

In [3]:
weather_data=pd.read_csv(weather_path)
weather_data.head()


,Unnamed: 0,City,Country,Longitude (Degrees),Latitude (Degrees),Maximum Temperature (C),Humidity (%),Wind Speed (km/h),Cloudiness (%)
0,0,São Filipe,CV,-24.50,14.90,27.74,70,4.04,2
1,1,Rikitea,PF,-134.97,-23.12,23.34,79,1.48,41
2,2,Berlevåg,NO,29.09,70.86,2.28,86,17.50,75
3,3,New Norfolk,AU,147.06,-42.78,11.39,79,0.46,100
4,4,Whittlesea,ZA,26.82,-32.18,28.40,26,15.24,99


In [4]:
weather_data.rename({'Longitude (Degrees)': 'Lng', 'Latitude (Degrees)': 'Lat'}, axis=1, inplace=True)

In [5]:
type(weather_data)

pandas.core.frame.DataFrame

In [6]:
weather_data.shape

(568, 9)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
gmaps.configure(api_key=g_key)

In [13]:
locations = weather_data[["Lat", "Lng"]].astype(float)
humidity = weather_data["Humidity (%)"].astype(float)

In [14]:
type(locations)

pandas.core.frame.DataFrame

In [15]:
locations.head()

,Lat,Lng
0,14.90,-24.50
1,-23.12,-134.97
2,70.86,29.09
3,-42.78,147.06
4,-32.18,26.82


In [16]:
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=3)
fig.add_layer(heatmap_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [17]:
type(weather_data)

pandas.core.frame.DataFrame

In [18]:
hotel_df=weather_data.dropna()

In [19]:
type(hotel_df)

pandas.core.frame.DataFrame

In [20]:
hotel_df.head()

,Unnamed: 0,City,Country,Lng,Lat,Maximum Temperature (C),Humidity (%),Wind Speed (km/h),Cloudiness (%)
0,0,São Filipe,CV,-24.50,14.90,27.74,70,4.04,2
1,1,Rikitea,PF,-134.97,-23.12,23.34,79,1.48,41
2,2,Berlevåg,NO,29.09,70.86,2.28,86,17.50,75
3,3,New Norfolk,AU,147.06,-42.78,11.39,79,0.46,100
4,4,Whittlesea,ZA,26.82,-32.18,28.40,26,15.24,99


In [21]:
type(hotel_df)

pandas.core.frame.DataFrame

In [22]:
hotel_df=hotel_df[hotel_df["Maximum Temperature (C)"] >25]

In [23]:
hotel_df=hotel_df[hotel_df["Wind Speed (km/h)"] <10]


In [24]:
hotel_df.shape

(165, 9)

In [25]:
type(hotel_df)

pandas.core.frame.DataFrame

In [26]:
hotel_df.shape

(165, 9)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [27]:
hotel_df['Hotel'] = ""

In [28]:
hotel_df.head()

,Unnamed: 0,City,Country,Lng,Lat,Maximum Temperature (C),Humidity (%),Wind Speed (km/h),Cloudiness (%),Hotel
0,0,São Filipe,CV,-24.50,14.90,27.74,70,4.04,2,
10,10,Sittwe,MM,92.90,20.15,27.69,84,8.94,95,
20,20,Entre Rios,BR,-38.08,-11.94,30.45,49,4.06,13,
31,31,Fort Myers Beach,US,-81.95,26.45,25.36,94,3.10,1,
34,34,Malakal,SS,31.66,9.53,33.38,48,1.67,82,


In [29]:
hotel_df.dtypes

Unnamed: 0                   int64
City                        object
Country                     object
Lng                        float64
Lat                        float64
Maximum Temperature (C)    float64
Humidity (%)                 int64
Wind Speed (km/h)          float64
Cloudiness (%)               int64
Hotel                       object
dtype: object

In [30]:
# find the closest hotel for each set of coordinates
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
  
 

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city= row["City"]

    params = {
    "location": f"{lat}, {lng}",
    "radius": 5000,
    'keyword': "Hotel",
    "key": g_key,
        }  

# assemble url and make API request
    print(f"Retrieving Results for {city} at coordinates ({lat}, {lng})")
    response = requests.get(base_url, params=params).json()
    
# extract results
    results = response['results']
    
    try:
        print(f"Closest hotel in {city} is {results[0]['name']}.")
        print("------------")
        hotel_df.loc[index, "Hotel"] = results[0]["name"]


    except :
        print("Missing field/result... skipping.")
        print("------------")

Retrieving Results for São Filipe at coordinates (14.9, -24.5)
Closest hotel in São Filipe is Hotel Ocean View & Restaurante Seafood.
------------
Retrieving Results for Sittwe at coordinates (20.15, 92.9)
Closest hotel in Sittwe is Yuzana Aung Hotel.
------------
Retrieving Results for Entre Rios at coordinates (-11.94, -38.08)
Closest hotel in Entre Rios is Hotel Rio Center.
------------
Retrieving Results for Fort Myers Beach at coordinates (26.45, -81.95)
Closest hotel in Fort Myers Beach is Hampton Inn & Suites Fort Myers Beach/Sanibel Gateway.
------------
Retrieving Results for Malakal at coordinates (9.53, 31.66)
Closest hotel in Malakal is Nile Palace Hotel.
------------
Retrieving Results for Port Alfred at coordinates (-33.59, 26.89)
Closest hotel in Port Alfred is Royal St Andrews Hotel.
------------
Retrieving Results for Richards Bay at coordinates (-28.78, 32.04)
Closest hotel in Richards Bay is BON Hotel Waterfront Richards Bay.
------------
Retrieving Results for Butar

Missing field/result... skipping.
------------
Retrieving Results for Amahai at coordinates (-3.33, 128.92)
Closest hotel in Amahai is Hotel Onemai.
------------
Retrieving Results for Pangoa at coordinates (-12.12, -73.0)
Missing field/result... skipping.
------------
Retrieving Results for Bontang at coordinates (0.13, 117.5)
Closest hotel in Bontang is Hotel Garuda Bontang.
------------
Retrieving Results for Kangaba at coordinates (11.93, -8.42)
Closest hotel in Kangaba is Hotel Residence KANGABA.
------------
Retrieving Results for Jacareacanga at coordinates (-6.22, -57.75)
Closest hotel in Jacareacanga is HOTEL VIANA.
------------
Retrieving Results for Goma at coordinates (-1.68, 29.22)
Closest hotel in Goma is Goma Serena Hotel.
------------
Retrieving Results for Boma at coordinates (7.08, -2.17)
Missing field/result... skipping.
------------
Retrieving Results for Bambous Virieux at coordinates (-20.34, 57.76)
Closest hotel in Bambous Virieux is Seaview Villa.
------------
R

Closest hotel in Eydhafushi is Soneva Fushi.
------------
Retrieving Results for Puri at coordinates (19.8, 85.85)
Closest hotel in Puri is Hotel Holiday Resort.
------------
Retrieving Results for Port Blair at coordinates (11.67, 92.75)
Closest hotel in Port Blair is Symphony Samudra, Port Blair.
------------
Retrieving Results for Turaif at coordinates (31.67, 38.66)
Closest hotel in Turaif is Golden Dune Hotel.
------------
Retrieving Results for Luanda at coordinates (-8.84, 13.23)
Closest hotel in Luanda is EPIC SANA Luanda.
------------
Retrieving Results for Kushālgarh at coordinates (23.2, 74.45)
Closest hotel in Kushālgarh is Hotel Janvi.
------------
Retrieving Results for Bikaner at coordinates (28.02, 73.3)
Closest hotel in Bikaner is Narendra Bhawan Bikaner (A Unit of MRS Hospitality).
------------
Retrieving Results for Sungai Raya at coordinates (0.7, 108.9)
Missing field/result... skipping.
------------
Retrieving Results for Banda Aceh at coordinates (5.56, 95.32)
Clo

In [31]:
hotel_df.head()

,Unnamed: 0,City,Country,Lng,Lat,Maximum Temperature (C),Humidity (%),Wind Speed (km/h),Cloudiness (%),Hotel
0,0,São Filipe,CV,-24.50,14.90,27.74,70,4.04,2,Hotel Ocean View & Restaurante Seafood
10,10,Sittwe,MM,92.90,20.15,27.69,84,8.94,95,Yuzana Aung Hotel
20,20,Entre Rios,BR,-38.08,-11.94,30.45,49,4.06,13,Hotel Rio Center
31,31,Fort Myers Beach,US,-81.95,26.45,25.36,94,3.10,1,Hampton Inn & Suites Fort Myers Beach/Sanibel ...
34,34,Malakal,SS,31.66,9.53,33.38,48,1.67,82,Nile Palace Hotel


In [32]:
hotel_df.shape

(165, 10)

In [33]:
#Filter out rows where we did not find a hotel name.

In [34]:
hotel_df=hotel_df[hotel_df["Hotel"] !=""]

In [35]:
hotel_df.shape

(136, 10)

In [36]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [37]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()])
fig.add_layer(markers)


In [38]:
fig

Figure(layout=FigureLayout(height='420px'))